In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from astropy.io import fits
from astropy.table import Table
from astropy.cosmology import Planck18 as cosmo
import astropy.units as u

In [ ]:
# Load candidate table
path_to_fits = '/home/alberto/cosmos/ista/COLA1/VISCHECK_COLA1_O3/O3_candidates_1.0.fits'
O3_cands = Table.read(path_to_fits).to_pandas()

# Load the full catalog
path_to_fits = '../catalogs/COLA1_photcat_v1_noisemodel_short.fits'
select_cols = ['NUMBER', 'F115W_AUTO_mag', 'F150W_AUTO_mag',
               'F200W_AUTO_mag', 'F356W_AUTO_mag',]
cola1_cat = Table.read(path_to_fits).to_pandas()[select_cols]
O3_cands.dtypes

In [ ]:
# Plot colors
fig, ax = plt.subplots()

ax.scatter(cola1_cat['F115W_AUTO_mag'] - cola1_cat['F150W_AUTO_mag'],
           cola1_cat['F150W_AUTO_mag'] - cola1_cat['F200W_AUTO_mag'],
           s=1)
ax.scatter(O3_cands['F115W_AUTO_mag'] - O3_cands['F150W_AUTO_mag'],
           O3_cands['F150W_AUTO_mag'] - O3_cands['F200W_AUTO_mag'],
           s=10)

ax.scatter((O3_cands['F115W_AUTO_mag'] - O3_cands['F150W_AUTO_mag'])[93],
           (O3_cands['F150W_AUTO_mag'] - O3_cands['F200W_AUTO_mag'])[93],
           s=100, marker='*', c='k')

ax.set_ylim(-10, 10)
ax.set_xlim(-10, 10)

ax.set_xlabel('F115W - F150W')
ax.set_ylabel('F150W - F200W')

plt.show()

In [ ]:
# Plot colors
fig, ax = plt.subplots()

ax.scatter(cola1_cat['F115W_AUTO_mag'] - cola1_cat['F150W_AUTO_mag'],
           cola1_cat['F200W_AUTO_mag'] - cola1_cat['F356W_AUTO_mag'],
           s=1)
ax.scatter(O3_cands['F115W_AUTO_mag'] - O3_cands['F150W_AUTO_mag'],
           O3_cands['F200W_AUTO_mag'] - O3_cands['F356W_AUTO_mag'],
           s=10)

ax.scatter((O3_cands['F115W_AUTO_mag'] - O3_cands['F150W_AUTO_mag'])[93],
           (O3_cands['F200W_AUTO_mag'] - O3_cands['F356W_AUTO_mag'])[93],
           s=100, marker='*', c='k')

ax.set_ylim(-5, 10)
ax.set_xlim(-5, 7.5)

ax.set_xlabel('F115W - F150W')
ax.set_ylabel('F200W - F356W')

plt.show()

In [ ]:
fig, ax = plt.subplots()

bins = np.linspace(-5, 5)
ax.hist(cola1_cat['F115W_AUTO_mag'] - cola1_cat['F150W_AUTO_mag'],
        bins, histtype='step', density=True, label='All')
ax.hist(O3_cands['F115W_AUTO_mag'] - O3_cands['F150W_AUTO_mag'],
        bins, histtype='step', density=True, label='Candidates')

ax.legend()

ax.set_xlabel('F115W - F150W')

plt.show()

#############

fig, ax = plt.subplots()

bins = np.linspace(-5, 5)
ax.hist(cola1_cat['F150W_AUTO_mag'] - cola1_cat['F200W_AUTO_mag'],
        bins, histtype='step', density=True, label='All')
ax.hist(O3_cands['F150W_AUTO_mag'] - O3_cands['F200W_AUTO_mag'],
        bins, histtype='step', density=True, label='Candidates')

ax.legend()

ax.set_xlabel('F150W - F200W')

plt.show()

#############

fig, ax = plt.subplots()

bins = np.linspace(-5, 5)
ax.hist(cola1_cat['F200W_AUTO_mag'] - cola1_cat['F356W_AUTO_mag'],
        bins, histtype='step', density=True, label='All')
ax.hist(O3_cands['F200W_AUTO_mag'] - O3_cands['F356W_AUTO_mag'],
        bins, histtype='step', density=True, label='Candidates')

ax.legend()

ax.set_xlabel('F200W - F356W')

plt.show()

In [ ]:
fig, ax = plt.subplots()

ax.hist(O3_cands['z_O3doublet'], bins=40)

ax.set_xlabel('z_O3doublet')

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))

# COLA1 coords.
COLA1_RA = 150.64751,
COLA1_DEC = 2.20375

draw_order = np.argsort(O3_cands['z_O3doublet'])
sc = ax.scatter(O3_cands['ALPHA_J2000_det'][draw_order], O3_cands['DELTA_J2000_det'][draw_order],
                c=O3_cands['z_O3doublet'][draw_order], cmap='gist_rainbow')

ax.scatter(COLA1_RA, COLA1_DEC, s=150, marker='*',
           color='k')

colorbar = fig.colorbar(sc)

# Distance at z=6.593
ang_diameter_dist = cosmo.angular_diameter_distance(6.593) / u.rad
ang_diameter_dist = ang_diameter_dist.to(u.kpc / u.deg)
ang_per_dist = (300 * u.kpc / ang_diameter_dist)
ax.errorbar(150.67, 2.16, xerr=ang_per_dist.value * 0.5,
            fmt='none', capsize=4, c='k', elinewidth=2,
            capthick=2)
ax.text(150.67, 2.156, '300 pMpc',
        horizontalalignment='center')

ax.set_aspect('equal')
ax.set_xlabel('RA')
ax.set_ylabel('DEC')
ax.set_ylim(2.15, 2.26)
ax.set_xlim(150.59, 150.70)

plt.show()

In [ ]:
from astropy.coordinates import SkyCoord

fig, ax = plt.subplots(figsize=(8, 8))

cdist0 = cosmo.comoving_distance(O3_cands['z_O3doublet'])
COLA1_dist = cosmo.comoving_distance(6.593)

coord_Arr = SkyCoord(np.array(O3_cands['ALPHA_J2000_det']) * u.deg,
                     np.array(O3_cands['DELTA_J2000_det']) * u.deg,
                     distance=cdist0)
coord_COLA1 = SkyCoord(COLA1_RA * u.deg, COLA1_DEC * u.deg,
                       distance=COLA1_dist)

dist_to_COLA1 = coord_COLA1.separation_3d(coord_Arr).to(u.Mpc)

mask = (dist_to_COLA1 < 1000 * u.Mpc)
sc = ax.scatter(O3_cands['ALPHA_J2000_det'][mask], O3_cands['DELTA_J2000_det'][mask],
                c=dist_to_COLA1[mask], cmap='viridis')

ax.scatter(COLA1_RA, COLA1_DEC, s=150, marker='*',
           color='k')

colorbar = plt.colorbar(sc)

ax.errorbar(150.67, 2.16, xerr=ang_per_dist.value * 0.5,
            fmt='none', capsize=4, c='k', elinewidth=2,
            capthick=2)
ax.text(150.67, 2.156, '300 pkpc',
        horizontalalignment='center')

ax.set_ylim(2.15, 2.26)
ax.set_xlim(150.59, 150.70)
ax.set_aspect('equal')
ax.set_xlabel('RA')
ax.set_ylabel('DEC')

plt.show()

In [ ]:
plt.hist(dist_to_COLA1, bins=50)

plt.xlabel('cMpc')

plt.show()